### Настройка Kaggle, загрузка данных и библиотек

In [1]:
!pip install --upgrade --force-reinstall --no-deps kaggle

     |████████████████████████████████| 61kB 2.8MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=fb6445f49a9e10ecf3efd0b76d2cf561fa743f04ad54e1066fc30057cb0a7104
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12


In [2]:
!pip install --upgrade catboost==0.25

     |████████████████████████████████| 67.3MB 47kB/s 


In [3]:
import os

user = '#####'
key = '#####'
 
if '.kaggle' not in os.listdir('/root'):
    !mkdir ~/.kaggle
!touch /root/.kaggle/kaggle.json
!chmod 666 /root/.kaggle/kaggle.json
with open('/root/.kaggle/kaggle.json', 'w') as f:
    f.write('{"username":"%s","key":"%s"}' % (user, key))
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c #####

In [ ]:
!unzip /content/##### -d data

### Imports

In [6]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from catboost import CatBoostRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_log_error
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler

### EDA

In [7]:
data = pd.read_csv('/content/data/train.csv').drop(['Id'], axis=1)

In [8]:
data

,Room,Price,Size,Balcony,Floor,FloorsTotal,Walls,Age,Lift,District,Okrug
0,5,210020000.0,198,балкон,1,2,NaN,вторичка,NaN,Сокол,САО
1,2,17630000.0,64,NaN,3,8,монолитный,вторичка,грузовой,Зюзино,ЮЗАО
2,5,142400000.0,161,балкон,7,7,кирпичный,вторичка,пассажирский,Пресненский,ЦАО
3,5,142520000.0,265,NaN,5,5,кирпичный,вторичка,NaN,Басманный,ЦАО
4,3,34070000.0,88,NaN,10,42,монолитный,вторичка,грузовой,Хорошевский,САО
...,...,...,...,...,...,...,...,...,...,...,...
40503,2,14100000.0,55,лоджия,17,18,монолитный,вторичка,пассажирский,Щукино,СЗАО
40504,2,6860000.0,44,балкон,2,5,панельный дом,вторичка,NaN,Северное Тушино,СЗАО
40505,3,32110000.0,96,NaN,14,44,кирпично-монолитный,вторичка,пассажирский,Щукино,СЗАО
40506,3,7760000.0,50,балкон,5,9,панельный дом,вторичка,пассажирский,Северное Тушино,СЗАО


In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40508 entries, 0 to 40507
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Room         40508 non-null  int64  
 1   Price        40508 non-null  float64
 2   Size         40508 non-null  int64  
 3   Balcony      27758 non-null  object 
 4   Floor        40508 non-null  int64  
 5   FloorsTotal  40508 non-null  int64  
 6   Walls        34623 non-null  object 
 7   Age          39967 non-null  object 
 8   Lift         31082 non-null  object 
 9   District     40151 non-null  object 
 10  Okrug        40151 non-null  object 
dtypes: float64(1), int64(4), object(6)
memory usage: 3.4+ MB


In [10]:
data.describe()

,Room,Price,Size,Floor,FloorsTotal
count,40508.000000,4.050800e+04,40508.000000,40508.000000,40508.000000
mean,2.217932,2.416226e+07,73.782636,8.286215,15.950059
std,1.086662,5.118166e+07,52.706547,7.441877,10.341203
min,1.000000,1.660000e+06,10.000000,0.000000,2.000000
25%,1.000000,7.100000e+06,42.000000,3.000000,9.000000
50%,2.000000,1.045000e+07,59.000000,6.000000,14.000000
75%,3.000000,1.953250e+07,80.000000,11.000000,19.000000
max,5.000000,2.321650e+09,1703.000000,90.000000,101.000000


In [14]:
train_prices = data['Price']
train = data.drop(columns=['Price'])

In [15]:
for i in train.select_dtypes(exclude=['float64', 'int64']).columns.values:
  print(len(train[i].unique()), train[i].unique())

3 ['балкон' nan 'лоджия']
6 [nan 'монолитный' 'кирпичный' 'панельный дом' 'кирпично-монолитный'
 'блочный']
3 ['вторичка' 'новостройка' nan]
3 [nan 'грузовой' 'пассажирский']
121 ['Сокол' 'Зюзино' 'Пресненский' 'Басманный' 'Хорошевский' 'Академический'
 'Красносельский' 'Раменки' 'Можайский' 'Хамовники' 'Вешняки' 'Митино'
 'Северное Чертаново' 'Жулебино' 'Щукино' 'Бутырский' 'Ясенево' 'Отрадное'
 'Кунцево' 'Измайлово Восточное' 'Измайлово' 'Арбат' 'Тверской' 'Донской'
 'Таганка' 'Якиманка' 'Медведково Южное' 'Мещанский' 'Нагорный' 'Куркино'
 'Очаково-Матвеевское' 'Ломоносовский' 'Кузьминки' 'Даниловский' 'Коптево'
 'Богородский' 'Головинский' 'Бибирево' 'Замоскворечье'
 'Хорошёво-Мневники' 'Марьина роща' 'Бабушкинский' 'Аэропорт' 'Печатники'
 'Останкино' 'Фили Давыдково' 'Свиблово' 'Марьино' 'Алексеевский'
 'Черемушки' 'Бирюлёво Восточное' 'Покровское Стрешнево' 'Новогиреево'
 'Перово' 'Бескудниковский' 'Северное Тушино' 'Филевский парк' 'Строгино'
 'Южное Бутово' 'Марфино' 'Савеловски

### Предобработка данных

In [16]:
train.dtypes

Room            int64
Size            int64
Balcony        object
Floor           int64
FloorsTotal     int64
Walls          object
Age            object
Lift           object
District       object
Okrug          object
dtype: object

In [17]:
print('float64 and int64:')
for i in list(train.select_dtypes(include=['float64', 'int64']).columns.values):
  print(i)
print('=============')
print('NOT float64 or int64:')
for i in train.select_dtypes(exclude=['float64', 'int64']).columns.values:
  print(i)

float64 and int64:
Room
Size
Floor
FloorsTotal
NOT float64 or int64:
Balcony
Walls
Age
Lift
District
Okrug


In [18]:
# Split dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(train, train_prices, test_size=0.2, random_state=42)


# define transformers
si_num = SimpleImputer(strategy='mean')
#si_num = SimpleImputer(strategy='constant', fill_value=-9999)
#si_cat = SimpleImputer(strategy='most_frequent')
si_cat = SimpleImputer(strategy='constant', fill_value='MISSING')
ohe = OneHotEncoder()

# define column groups with same processing
num_vars = list(train.select_dtypes(include=['float64', 'int64']).columns.values)
cat_vars = list(train.select_dtypes(exclude=['float64', 'int64']).columns.values)

# set up pipelines for each column group
categorical_pipe = Pipeline([('si_cat', si_cat), ('ohe', ohe)])
numeric_pipe = Pipeline([('si_num', si_num)])

# set up columnTransformer
column_transformer = ColumnTransformer(
                    transformers=[
                        ('nums', numeric_pipe, num_vars),
                        ('cats', categorical_pipe, cat_vars)
                    ],
                    remainder='drop'
                    )

# Transform the data
X_train = column_transformer.fit_transform(X_train)
X_test = column_transformer.transform(X_test)

### Создание и настройка модели

In [19]:
y_train_log = np.log(y_train)

In [20]:
# Define classifier with default parameters
model = CatBoostRegressor()


# Estimate grid of the classifier hyperparameters
parameters = {'learning_rate':[1.0, 0.7, 0.5, 0.3], 
              'max_depth' : [3, 7, 11],
              'n_estimators' : [250, 400, 500]}
              #'logging_level' : ['Silent']}

# Define GridSearch parameters
gs = GridSearchCV(model,                 # Classifier object to optimize
                  parameters,          # Grid of the hyperparameters
                  scoring='neg_mean_squared_error',  # Claasification quality metric to optimize
                  cv=3)                # Number of folds in KFolds cross-validation (CV)


# Run Grid Search optimization
gs.fit(X_train, y_train_log)
gs.best_params_

Выходные данные были обрезаны до нескольких последних строк (5000).
450:	learn: 0.1417877	total: 3.13s	remaining: 341ms
451:	learn: 0.1416791	total: 3.14s	remaining: 334ms
452:	learn: 0.1415767	total: 3.15s	remaining: 327ms
453:	learn: 0.1415354	total: 3.15s	remaining: 320ms
454:	learn: 0.1414734	total: 3.16s	remaining: 313ms
455:	learn: 0.1413974	total: 3.17s	remaining: 306ms
456:	learn: 0.1412989	total: 3.17s	remaining: 299ms
457:	learn: 0.1412445	total: 3.18s	remaining: 292ms
458:	learn: 0.1411603	total: 3.19s	remaining: 285ms
459:	learn: 0.1410118	total: 3.19s	remaining: 278ms
460:	learn: 0.1409501	total: 3.2s	remaining: 271ms
461:	learn: 0.1409231	total: 3.2s	remaining: 264ms
462:	learn: 0.1408203	total: 3.21s	remaining: 256ms
463:	learn: 0.1407275	total: 3.21s	remaining: 249ms
464:	learn: 0.1405828	total: 3.22s	remaining: 243ms
465:	learn: 0.1404953	total: 3.23s	remaining: 236ms
466:	learn: 0.1403971	total: 3.23s	remaining: 229ms
467:	learn: 0.1403324	total: 3.24s	remaining: 222m

{'learning_rate': 0.3, 'max_depth': 7, 'n_estimators': 500}

In [21]:
gs.best_score_

-0.03698166359939783

In [22]:
mean_squared_log_error(y_test, np.exp(gs.predict(X_test)))

0.03487731959220775

### Финальная модель

In [23]:
final_train = column_transformer.transform(train)
train_prices_log = np.log(train_prices)


final_model = CatBoostRegressor(learning_rate=0.3, max_depth=7, n_estimators=500, logging_level='Silent')
final_model.fit(final_train, train_prices_log)

### Загрузка решения

In [24]:
sample_submission = pd.read_csv('/content/data/sample_submission.csv')

In [25]:
sample_submission

,Id,Price
0,40509,1.236817e+07
1,40510,3.865640e+06
2,40511,1.082091e+07
3,40512,8.961175e+06
4,40513,1.410078e+07
...,...,...
10122,50631,5.746093e+06
10123,50632,5.889010e+06
10124,50633,7.293338e+06
10125,50634,9.915968e+06


In [26]:
test_data = pd.read_csv('/content/data/test.csv')

In [27]:
test_data

,Id,Room,Size,Balcony,Floor,FloorsTotal,Walls,Age,Lift,District,Okrug
0,40509,2,58,NaN,2,5,NaN,вторичка,NaN,Богородский,ВАО
1,40510,5,266,лоджия,3,14,NaN,вторичка,грузовой,Мещанский,ЦАО
2,40511,5,197,лоджия,5,9,NaN,вторичка,грузовой,Басманный,ЦАО
3,40512,5,233,лоджия,3,14,NaN,вторичка,грузовой,Мещанский,ЦАО
4,40513,2,69,NaN,2,6,NaN,вторичка,пассажирский,Красносельский,ЦАО
...,...,...,...,...,...,...,...,...,...,...,...
10122,50631,3,86,лоджия,14,14,панельный дом,вторичка,грузовой,Люблино,ЮВАО
10123,50632,3,82,лоджия,6,17,панельный дом,вторичка,грузовой,Северное Бутово,ЮЗАО
10124,50633,1,40,лоджия,3,23,монолитный,вторичка,грузовой,Южное Бутово,ЮЗАО
10125,50634,2,57,NaN,3,9,кирпичный,вторичка,пассажирский,Даниловский,ЮАО


In [28]:
result = test_data['Id']
test_feat = test_data.drop(columns=['Id'])

In [29]:
test_feat

,Room,Size,Balcony,Floor,FloorsTotal,Walls,Age,Lift,District,Okrug
0,2,58,NaN,2,5,NaN,вторичка,NaN,Богородский,ВАО
1,5,266,лоджия,3,14,NaN,вторичка,грузовой,Мещанский,ЦАО
2,5,197,лоджия,5,9,NaN,вторичка,грузовой,Басманный,ЦАО
3,5,233,лоджия,3,14,NaN,вторичка,грузовой,Мещанский,ЦАО
4,2,69,NaN,2,6,NaN,вторичка,пассажирский,Красносельский,ЦАО
...,...,...,...,...,...,...,...,...,...,...
10122,3,86,лоджия,14,14,панельный дом,вторичка,грузовой,Люблино,ЮВАО
10123,3,82,лоджия,6,17,панельный дом,вторичка,грузовой,Северное Бутово,ЮЗАО
10124,1,40,лоджия,3,23,монолитный,вторичка,грузовой,Южное Бутово,ЮЗАО
10125,2,57,NaN,3,9,кирпичный,вторичка,пассажирский,Даниловский,ЮАО


In [30]:
test_feat = column_transformer.transform(test_feat)

In [31]:
result_prices = np.exp(final_model.predict(test_feat))

In [32]:
result_prices

array([8.79581612e+06, 1.61285250e+08, 9.80196945e+07, ...,
       5.71293417e+06, 1.13182799e+07, 8.57529793e+06])

In [33]:
result_df = pd.concat([result, pd.Series(data=result_prices, name='Price')], axis=1)

In [34]:
result_df

,Id,Price
0,40509,8.795816e+06
1,40510,1.612852e+08
2,40511,9.801969e+07
3,40512,1.179843e+08
4,40513,2.554578e+07
...,...,...
10122,50631,1.144402e+07
10123,50632,1.227329e+07
10124,50633,5.712934e+06
10125,50634,1.131828e+07


In [35]:
result_df.to_csv('result3.csv', index=False)

In [ ]:
!kaggle competitions submit ##### -f result3.csv -m "CatBoost solution 3"